In [25]:
import pandas as pd
import numpy as np
pd.set_option("display.max_column", None)
from datetime import datetime

In [29]:
def generation_df_movies(path='../data/rotten_tomatoes_top_movies.csv') -> pd.core.frame.DataFrame:
    
    df_top_movies = pd.read_csv(path, sep=',')
    
    df_top_movies.drop(columns=['Unnamed: 0', 'synopsis', 'consensus', 'rating', 'writer', 'sound_mix', 'crew', 'link', 
                                'aspect_ratio', 'production_co', 'view_the_collection', 'producer', 'runtime','type'], inplace=True)
    
    df_top_movies.rename(columns={'release_date_(theaters)':'release_date_theaters'},inplace=True)
    
    df_top_movies['release_date_theaters'] = df_top_movies['release_date_theaters'].str.replace(' wide','',regex=True).replace(' limited','',regex=True).replace(',','', regex=True)
    df_top_movies['release_date_theaters'] = pd.to_datetime(df_top_movies['release_date_theaters'],format='%b %d %Y')
    df_top_movies = df_top_movies[(df_top_movies['box_office_(gross_usa)'].notna()) & (df_top_movies['release_date_theaters'] > '2000-01-01')].copy()
    
    df_top_movies.loc[df_top_movies['title'] == 'A Quiet Place', 'total_reviews'] = 384
    
    df_top_movies = df_top_movies.drop_duplicates(['title', 'year', 'critic_score', 'people_score', 'total_reviews',
           'total_ratings', 'genre', 'original_language', 'director',
           'release_date_theaters', 'release_date_(streaming)',
           'box_office_(gross_usa)']).reset_index(drop=True).copy()
    
    df_top_movies['id'] = range(1, df_top_movies.index.stop+1)
    
    df_top_movies = df_top_movies[['id','title', 'year', 'critic_score', 'people_score', 'total_reviews',
           'total_ratings', 'genre', 'original_language', 'director',
           'release_date_theaters', 'release_date_(streaming)',
           'box_office_(gross_usa)']]
    
    return df_top_movies

In [30]:
df_top_movies = generation_df_movies()

In [35]:
def dim_title() -> pd.core.frame.DataFrame:
    
    df_top_movies = generation_df_movies()
    dim_title = pd.DataFrame({'title':list(df_top_movies['title'].unique())})
    dim_title['id'] = range(1, dim_title.index.stop+1)
    dim_title = dim_title[['id','title']]
    return dim_title

In [41]:
df_title = dim_title()

In [36]:
def dim_genre() -> pd.core.frame.DataFrame:
    
    df_top_movies = generation_df_movies()
    df_top_movies['genre'] = df_top_movies['genre'].str.split(',')
    df_genre = pd.DataFrame({'genre':df_top_movies[['genre']].explode("genre").reset_index(drop=True).copy()['genre'].str.strip().unique()})
    df_genre['id'] = range(1,df_genre.index.stop + 1)
    df_genre = df_genre[['id','genre']]
    return df_genre

In [43]:
df_genre = dim_genre()

In [54]:
def rel_top_moviesWgenre() -> pd.core.frame.DataFrame:
    
    df_top_movies = generation_df_movies()
    df_genre = dim_genre()
    dict_genre = dict(zip(df_genre['genre'], df_genre['id']))
    df_top_movies['genre'] = df_top_movies['genre'].str.split(',')
    df_top_moviesWgenre = df_top_movies[['id','genre']].explode('genre')
    df_top_moviesWgenre['genre'] = df_top_moviesWgenre['genre'].str.strip()
    df_top_moviesWgenre.replace({'genre':dict_genre},inplace=True)
    return df_top_moviesWgenre

In [55]:
df_top_moviesWgenre = rel_top_moviesWgenre()

In [57]:
def dim_language() -> pd.core.frame.DataFrame:
    
    df_top_movies = generation_df_movies()
    df_language = pd.DataFrame({'original_language':list(df_top_movies['original_language'].unique())})
    df_language['id'] = range(1, df_language.index.stop+1)
    df_language = df_language[['id','original_language']]
    return df_language

In [58]:
df_language = dim_language()

In [67]:
def dim_director() -> pd.core.frame.DataFrame:
    
    df_top_movies = generation_df_movies()
    df_top_movies['director'] = df_top_movies['director'].str.split(',')
    df_director = pd.DataFrame({'director':df_top_movies[['director']].explode("director").reset_index(drop=True).copy()['director'].str.strip().unique()})
    df_director['id'] = range(1,df_director.index.stop + 1)
    df_director = df_director[['id','director']]
    return df_director

In [68]:
df_director = dim_director()

In [107]:
def rel_top_moviesWdirector() -> pd.core.frame.DataFrame:
    df_top_movies = generation_df_movies()
    df_director = dim_director()
    dict_director = dict(zip(df_director['director'], df_director['id']))
    df_top_movies['director'] = df_top_movies['director'].str.split(',')
    df_top_moviesWdirector = df_top_movies[['id','director']].explode('director')
    df_top_moviesWdirector['director'] = df_top_moviesWdirector['director'].str.strip()
    df_top_moviesWdirector.replace({'director':dict_director},inplace=True)
    df_top_moviesWdirector.rename(columns={'director':'director_id'},inplace=True)
    return df_top_moviesWdirector

In [108]:
df_top_moviesWdirector = rel_top_moviesWdirector()

In [122]:
create_foreing_key(df_top_movies, df_language, 'original_language')

,id,title,year,critic_score,people_score,total_reviews,total_ratings,genre,director,release_date_theaters,release_date_(streaming),box_office_(gross_usa),original_language_id
0,1,Black Panther,2018,96,79.0,519,"50,000+","adventure, action, fantasy",[Ryan Coogler],2018-02-16,"May 2, 2018",$700.2M,1
1,2,Avengers: Endgame,2019,94,90.0,538,"50,000+","sci fi, adventure, action, fantasy","[Anthony Russo, Joe Russo]",2019-04-26,"Jul 30, 2019",$858.4M,1
2,3,Mission: Impossible -- Fallout,2018,97,88.0,433,"10,000+","action, mystery and thriller, adventure",[Christopher McQuarrie],2018-07-27,"Nov 20, 2018",$220.1M,1
3,4,Mad Max: Fury Road,2015,97,86.0,427,"100,000+","adventure, action",[George Miller],2015-05-15,"Aug 10, 2016",$153.6M,1
4,5,Spider-Man: Into the Spider-Verse,2018,97,93.0,387,"10,000+","action, adventure, fantasy, comedy, kids and f...","[Bob Persichetti, Peter Ramsey, Rodney Rothman]",2018-12-14,"Mar 7, 2019",$190.2M,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,230,Pain and Glory,2019,96,91.0,293,250+ Verified,drama,[Pedro Almodóvar],2019-10-04,"Nov 8, 2019",$4.6M,21
453,248,Pan's Labyrinth,2006,95,91.0,238,"250,000+","drama, war, fantasy",[Guillermo del Toro],2006-12-29,"Aug 1, 2010",$37.6M,21
454,321,Cold War,2018,92,81.0,248,"1,000+","drama, history, romance",[Pawel Pawlikowski],2018-12-21,"Mar 31, 2019",$4.6M,22
455,323,The Lunchbox,2013,97,87.0,117,"10,000+","drama, romance",[Ritesh Batra],2014-02-28,"Dec 8, 2015",$4.2M,23
